# Natural Language Processing

## Exercise Sheet 4

In [1]:
#imports for all exercises
import pprint
import re
import nltk
from nltk.corpus import gutenberg
import networkx as nx
from nltk.corpus import wordnet as wn
from nltk import memoize
import timeit

ModuleNotFoundError: No module named 'nltk.corpus'

### Exercise 1

Write a program to initialize a two-dimensional array of sets called `word_vowels` and process a list of words, adding each word to `word_vowels[l][v]` where $l$ is the length of the word and $v$ is the number of vowels it contains. Test your program with a 10x10-array and the list `['Alice', 'hat', 'heute', 'ihren', 'freien', 'Tag']`.

In [ ]:
words = ['Alice', 'hat', 'heute', 'ihren', 'freien', 'Tag']
vowels = [[v for v in word if v in 'aeiou'] for word in words]
word_vowels = []
for (l,v) in zip(words, vowels):
    word_vowels.append([len(l), len(v)])

word_vowels

### Exercise 2

Write a program that prints all words that only appear in the last 10\% of a text. Test your code with the file `'shakespeare-macbeth.txt'` from the Gutenberg Corpus.


In [ ]:
def return_last_10(text):
    cut = int(0.9 * len(text))
    return text[cut:]
text = gutenberg.words()
print(return_last_10(text))

### Exercise 3

Write a program that takes a sentence expressed as a single string, splits it and counts up the words. Get it to print out each word and the word's frequency, one per line, in alphabetical order. Test it with the sentence: `'das ist heute wieder einmal wirklich ein sehr schöner tag das kann ich dir wieder einmal sagen'`.


In [ ]:
def text_freq(raw):
    text = sorted(nltk.word_tokenize(raw))
    count = len(text)
    return dict(nltk.FreqDist(text))

raw = 'das ist heute wieder einmal wirklich ein sehr schöner tag das kann ich dir wieder einmal sagen'
text_freq(raw)

### Exercise 4

Write a function `sort_dist(candidates, target)`. The `candidates` are a list of strings representing WordNet synset names, and `target` a synset name string. The function shall sort the `candidates` for proximity to the `target` synset using `shortest_path_distance()`. 

Test your function with `candidates=['minke_whale.n.01','orca.n.01','novel.n.01', 'tortoise.n.01']` and `target='right_whale.n.01'`.

In [ ]:
def sort_dist(candidates, target):
    target = wn.synset(target)
    dic = {candidate:target.shortest_path_distance(wn.synset(candidate)) 
    for candidate in candidates}
    return dict(sorted(dic.items(), key=lambda item: item[1]))

candidates=['minke_whale.n.01','orca.n.01','novel.n.01', 'tortoise.n.01']
target='right_whale.n.01'
sort_dist(candidates, target)


### Exercise 5

Write a recursive function `lookup(trie, key)` that looks up a `key` in a `trie`, and returns the value it finds. The function should cover the following cases:

a) it should return a corresponding message if the key is not included in the trie;  
b) it should return a message if the key is not unique, i.e. if there are several words for this prefix;  
c) if a word is uniquely determined by the key prefix it should be returned as result. 

Try your function for the following trie and test cases:

In [ ]:
def insert(trie, key, value):
    if key:
        first, rest = key[0], key[1:]
        if first not in trie:
            trie[first] = {}
        insert(trie[first], rest, value)
    else:
        trie['value'] = value

trie = {}
insert(trie, 'chat', 'cat')
insert(trie, 'chien', 'dog')
insert(trie, 'chair', 'flesh')
insert(trie, 'chic', 'stylish')
insert(trie, 'cheval','horse') 
trie = dict(trie)             
pprint.pprint(trie, width=40)

In [ ]:
def lookup(trie, key):
    if key:
        first, rest = key[0], key[1:]
        if first not in trie:
            return 'key not included'
        return lookup(trie[first], rest)
    elif len(trie) > 1:
        return 'not unique'
    elif 'value' in trie.keys():
        return trie['value']
    else:
        while ('value' not in trie.keys()):
            [[ _,trie ]] = trie.items() # not so elegant way of reducing trie
        return trie['value']



In [ ]:
print(lookup(trie, 'chat'))
print(lookup(trie, 'cha'))
print(lookup(trie, 'souris'))
print(lookup(trie, 'cheval'))
print(lookup(trie, 'che'))
print(lookup(trie, 'chev'))

### Exercise 6

Write a recursive function `pp_trie` that pretty prints a trie in alphabetically sorted order by replacing common prefixes with `'-'` characters.
Test your implementation with the following example data:

In [ ]:
trie = {}
insert(trie, 'chat', 'cat')
insert(trie, 'souris', 'mouse')
insert(trie, 'chien', 'dog')
insert(trie, 'chair', 'flesh')
insert(trie, 'chic', 'stylish')
insert(trie, 'cheval','horse') 
trie = dict(trie)             
pprint.pprint(trie, width=40)

In [ ]:
# with help from stackoverflow
def pp_trie(trie, s = ""):
    first = True
    for k, v in sorted(trie.items(), key = lambda x: x[0]):
        if isinstance(v, dict):
            if first:
                prefix = s + k
                first = False
            else:
                prefix = '-'*len(s) + k
            pp_trie(v, prefix)
        else:
            stuff = s + ": " + v
            pprint.pprint(stuff, width=40)


In [ ]:
pp_trie(trie)

    chair: flesh
    ---t: cat
    --eval: horse
    --ic: stylish
    ---en: dog
    souris: mouse

### Exercise 7

The *Catalan numbers* arise in many applications of combinatorial mathematics, including the counting of parse trees. The series can be defined as follows: $C_0 = 1$, and $C_{n+1} = \sum_{i=0}^{n}C_iC_{n-i}$ for $n\geq{0}$.

Write:

a) a recursive function `cn(n)` to compute the $n$th Catalan number $C_{n}$,  
b) a corresponding function `cn2(n)` that uses dynamic programming by storing calculated solutions in a lookup table,  
c) a function `cn3(n)`, which is identical to `cn(n)` but uses a `memoize` decorator.

Test your functions first by calculating the Catalan numbers $C_0\dots C_{16}$ and then by using the `timeit` module:

In [ ]:
def cn(n):
    if n == 0:
        return 1
    else:
        return sum([cn(i)*cn(n-1-i) for i in range(n)])

def cn2(n):
    C = [0 for i in range(n+1)]
    C[0] = 1
    for i in range(1,n+1):
        C[i] = sum([C[j]*C[i-j-1] for j in range(i)])
    return C[n]

@memoize
def cn3(n):
    if n == 0:
        return 1
    else:
        return sum([cn(i)*cn(n-1-i) for i in range(n)])

In [ ]:
print('recursive: ', timeit.timeit("cn(16)", setup="from __main__ import cn", number=5))
print('DP: ', timeit.timeit("cn2(16)", setup="from __main__ import cn2", number=5))
print('memoize:', timeit.timeit("cn3(16)", setup="from __main__ import cn3", number=5))